In [1]:
import huxt as H
import huxt_analysis as HA
import huxt_inputs as Hin
import SIR_HUXt_plots as sirplt
import sir_huxt_mono_obs as sir

import astropy.units as u
from astropy.time import Time
import glob
import os
import h5py
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import scipy.stats as st
import sunpy.coordinates.sun as sn

from sklearn.neighbors import KernelDensity
from astropy.time import Time

In [2]:
start_time = Time('2008-01-01T00:00:00')
cr_num = np.fix(sn.carrington_rotation_number(start_time))
ert = H.Observer('EARTH', start_time)
vr_in = np.zeros(128) + 400*u.km/u.s

# Set up HUXt for a 3.5 day simulation, outputting every dt_scale 
model = H.HUXt(v_boundary=vr_in, cr_num=cr_num, cr_lon_init=ert.lon_c, latitude=ert.lat.to(u.deg),
               lon_start=270*u.deg, lon_stop=90*u.deg, simtime=4*u.day, dt_scale=4)

# Generate a "truth" CME
base_cme = sir.get_base_cme()

# Get HUXt solution of this truth CME, and observations from L5
model.solve([base_cme])
cme = model.cmes[0]

for lon in [-60, -50, -40, -30, -20]:
    obs1_lon = lon*u.deg
    Obs = sir.Observer(model, cme, obs1_lon, el_min=4.0, el_max=30.0)

    fig, ax = sirplt.plot_huxt_with_observer(model.time_out[16], model, [Obs], add_flank=True, add_fov=True)
    name = 'test_plot_osse_{:03d}.png'.format(lon+360)
    fig.savefig(name)
    plt.close('all')

In [3]:
# MONO OBSERVER MANY LONS
# Test the SIR scheme
np.random.seed(19630802)

model, model1d = sir.setup_huxt(dt_scale=20)

# Generate a "truth" CME
base_cme = sir.get_base_cme()

# Get HUXt solution of this truth CME, and observations from L5
model.solve([base_cme])
cme_truth = model.cmes[0]

#############
model1d.solve([base_cme])
cme_arr = model1d.cmes[0]
arrival_stats = cme_arr.compute_arrival_at_body('EARTH')
hit = arrival_stats['hit']
if arrival_stats['hit']:
    t_transit = arrival_stats['t_transit'].to(u.s).value
    v_hit = arrival_stats['v'].value
else:
    t_transit = np.NaN
    v_hit = np.NaN
###############

obs_lon = -60*u.deg
L5Obs = sir.Observer(model, cme_truth, obs_lon, el_min=4.0, el_max=30.0)

fig, ax = sirplt.plot_huxt_with_observer(model.time_out[8], model, [L5Obs], add_flank=True, add_fov=True)
fig.savefig('truth_L5.png')
plt.close('all')

dirs = sir.get_project_dirs()
output_dir = 'sir_multi_obs_test'
output_dir = os.path.join(dirs['sir_analysis'], output_dir)
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

# Run the SIR scheme on this event many times to see how the performance is
n_ens = 5

# Make a guess at the CME initial values 
cme_guess = sir.perturb_cme(base_cme)

# Low observational error
observed_cme_flank = L5Obs.compute_synthetic_obs(el_spread=0.1, cadence=1, el_min=4.0, el_max=30.0)

obs = {'observer_lon':obs_lon, 'observed_cme_flank':observed_cme_flank, 'truth_cme_params':cme_truth.parameter_array(),
       't_transit':t_transit, 'v_hit':v_hit}

tag = "mono_obs_run_{:03d}".format(0)
sir.SIR(model, model1d, cme_guess, obs, n_ens, output_dir, tag)

